In [1]:
from pyspark.sql import (
    functions as f,
    Row,
    SparkSession,
    types as t
)

spark = SparkSession.builder.appName("df_most_popular").getOrCreate()

In [2]:
# https://www.kaggle.com/datasets/csanhueza/the-marvel-universe-social-network
csv_file_path = "file:///home/jovyan/work/hero-network.csv"

df = spark.read\
            .option("header", "true")\
            .option("inferSchema", "true").csv(csv_file_path)
df.printSchema()
df.show(truncate=False)

root
 |-- hero1: string (nullable = true)
 |-- hero2: string (nullable = true)

+--------------------+--------------------+
|hero1               |hero2               |
+--------------------+--------------------+
|LITTLE, ABNER       |PRINCESS ZANDA      |
|LITTLE, ABNER       |BLACK PANTHER/T'CHAL|
|BLACK PANTHER/T'CHAL|PRINCESS ZANDA      |
|LITTLE, ABNER       |PRINCESS ZANDA      |
|LITTLE, ABNER       |BLACK PANTHER/T'CHAL|
|BLACK PANTHER/T'CHAL|PRINCESS ZANDA      |
|STEELE, SIMON/WOLFGA|FORTUNE, DOMINIC    |
|STEELE, SIMON/WOLFGA|ERWIN, CLYTEMNESTRA |
|STEELE, SIMON/WOLFGA|IRON MAN/TONY STARK |
|STEELE, SIMON/WOLFGA|IRON MAN IV/JAMES R.|
|STEELE, SIMON/WOLFGA|RAVEN, SABBATH II/EL|
|RAVEN, SABBATH II/EL|FORTUNE, DOMINIC    |
|RAVEN, SABBATH II/EL|ERWIN, CLYTEMNESTRA |
|RAVEN, SABBATH II/EL|IRON MAN/TONY STARK |
|RAVEN, SABBATH II/EL|IRON MAN IV/JAMES R.|
|IRON MAN IV/JAMES R.|FORTUNE, DOMINIC    |
|IRON MAN IV/JAMES R.|ERWIN, CLYTEMNESTRA |
|IRON MAN IV/JAMES R.|IRON MAN/TONY STAR